In [10]:
# pip install firebase_admin
#pip install datetime

In [6]:
import firebase_admin
from firebase_admin import credentials, firestore

In [7]:
cred = credentials.Certificate("./infinite-hope-444810-i6-firebase-adminsdk-yd3a1-b3d8b4f2ba.json")
firebase_admin.initialize_app(cred)

db = firestore.client()

In [46]:
import json 

# get boundary coordinates from the database
def fetch_coordinates():
    docs = db.collection('areas').stream()
    coordinates_per_area = {}

    for doc in docs:
        area = doc.to_dict()
        doc_id = doc.id
                
        markers = area.get('markers')
        list_markers = []
        
        if isinstance(markers, str):
            markers_lat_lon = json.loads(markers)
            list_markers.append(markers_lat_lon)
            
        else : 
            if isinstance(markers, object):
            
                for m in markers : 
                    geopoint = m.get('geopoint')
                    latitude = geopoint.latitude
                    longitude =geopoint.longitude
                    
                    if latitude is not None and longitude is not None:
                        list_markers.append([latitude,longitude])

        coordinates_per_area[doc_id] = list_markers
    return coordinates_per_area



coordinates_per_area = fetch_coordinates()
print(coordinates_per_area)
# could also be getting the data from the database 
boundary_coords = [
    [47.53824293542552, 9.400395601987839],
    [47.53819495042763, 9.401967376470566],
    [47.53864967442201, 9.400986023247242],
    [47.53824293542552, 9.400395601987839]
]


{'0Pe3rRyTvS3xY8H83EZL': [[47.53778549174926, 9.40149262547493], [47.53819495042763, 9.401967376470566], [47.53864967442201, 9.400986023247242], [47.53824293542552, 9.400395601987839]], '1lGgtnR9NeObZFqfVdNg': [[47.54536459911934, 9.397201426327229], [47.54474811757352, 9.396118819713593], [47.54428892011002, 9.396947287023067], [47.54502376990978, 9.397806264460087]], 'F6btsazl2hwVzpOz4sua': [[[47.519662668364006, 9.435665644705296], [47.520149705039586, 9.43602304905653], [47.51992056514916, 9.43639487028122], [47.51951073735549, 9.436104856431484]]], 'G6gJEQkIRZdCVIGf7izR': [[[47.53802224934071, 9.401967376470566], [47.53765941676388, 9.401542246341705], [47.53739436405552, 9.401838630437851], [47.53767729815369, 9.402696937322617], [47.53807317698121, 9.402465932071209]]], 'cuvoFD9ncu06XrWZ2OC4': [[[47.55004025339046, 9.385452345013618], [47.55053945401707, 9.386408887803555], [47.5509449661665, 9.385818131268024]]]}


In [16]:
import datetime
import random 

# generating realalistic data
# Update the GPS data generation to limit the total time span to 5 minutes
def generate_limited_gps_data(total_duration_minutes=5, interval_seconds=30):
    gps_data = []
    base_time = datetime.datetime.now()
    lat_min = min(coord[0] for coord in boundary_coords)
    lat_max = max(coord[0] for coord in boundary_coords)
    lon_min = min(coord[1] for coord in boundary_coords)
    lon_max = max(coord[1] for coord in boundary_coords)

    statuses = ["driving", "mowing", "calculating direction", "awaiting instructions"]
    
    total_seconds = total_duration_minutes * 60
    num_points = total_seconds // interval_seconds

    for i in range(num_points):
        # Generate a random coordinate within the boundary
        latitude = random.uniform(lat_min, lat_max)
        longitude = random.uniform(lon_min, lon_max)
        
        # Simulate timestamps and movements
        timestamp = base_time + datetime.timedelta(seconds=i * interval_seconds)
        time_spent = random.randint(10, interval_seconds)  # seconds spent at coordinate
        annotations = random.choice(statuses)
        
        gps_data.append({
            "Latitude": round(latitude, 8),
            "Longitude": round(longitude, 8),
            "Timestamp": timestamp.isoformat(),
            "Time Spent": time_spent,
            "Annotations": annotations
        })
    
    return gps_data

In [24]:
# write the data to firestore
def write_data_to_firestore(current_time, data):
    
    doc_ref = db.collection("session").document()
    doc_ref.set({
        'session_time' : current_time,
        'datapoints' : data
    })
    

In [25]:
# combining ressources to write (fake) data to firestore
limited_gps_data = generate_limited_gps_data(total_duration_minutes=5, interval_seconds=30)
time = datetime.datetime.now()

write_data_to_firestore(time, limited_gps_data)
